<a href="https://colab.research.google.com/github/anuanmariya/pcb_ml_project/blob/main/pcb_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu,True)
len(gpus)

0

In [ ]:
from google.colab import drive
drive.mount

<function google.colab.drive.mount(mountpoint, force_remount=False, timeout_ms=120000, readonly=False)>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

def load_dataset(path,
                 img_size=(160, 160),
                 batch_size=64,
                 shuffle = None):
    ds = tf.keras.utils.image_dataset_from_directory(
        path,
        seed=123,
        shuffle = True,
        image_size=img_size,
        batch_size=batch_size,
        labels="inferred",
        label_mode="int",
        color_mode='rgb'
    )

    class_names = ds.class_names
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
    return ds , class_names

In [ ]:
train_b,binary_class_names = load_dataset('/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/train')
val_b = load_dataset('/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/val')
test_b = load_dataset('/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/test')
train_m, multiclass_class_names= load_dataset('/content/drive/MyDrive/PCB_MULTICLASS_SPLIT/train')
val_m = load_dataset('/content/drive/MyDrive/PCB_MULTICLASS_SPLIT/val')
test_m = load_dataset('/content/drive/MyDrive/PCB_MULTICLASS_SPLIT/test')

Found 559 files belonging to 2 classes.
Found 159 files belonging to 2 classes.
Found 82 files belonging to 2 classes.
Found 7503 files belonging to 6 classes.
Found 1181 files belonging to 6 classes.
Found 1125 files belonging to 6 classes.


In [ ]:
def augment_and_autotune(dataset):
    AUTOTUNE = tf.data.AUTOTUNE

    def augment(image, label):
        def aug_fn(img):
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.7, upper=1.3)
            img = tf.clip_by_value(img, 0.0, 1.0)
            return img

        # If batched input, map over batch
        image = tf.map_fn(aug_fn, image)
        return image, label
    return dataset.map(augment, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)





In [ ]:
train_b = augment_and_autotune(train_b)
train_m = augment_and_autotune(train_m)

# Unpack the tuples before prefetching
val_b, _ = val_b
val_m, _ = val_m
test_b, _ = test_b
test_m, _ = test_m

val_b = val_b.prefetch(tf.data.AUTOTUNE)
val_m = val_m.prefetch(tf.data.AUTOTUNE)
test_b = test_b.prefetch(tf.data.AUTOTUNE)
test_m = test_m.prefetch(tf.data.AUTOTUNE)

Binary Classifier

In [ ]:
from tensorflow.keras import layers, models

def build_binary_model():
    model = models.Sequential([
        layers.Input(shape=(160, 160, 3)),
        layers.Rescaling(1./255),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')  # binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
model = build_binary_model()
history_b = model.fit(
    train_b,
    validation_data=val_b,
    epochs=5
)

Epoch 1/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 105s 10s/step - accuracy: 0.8605 - loss: 0.5638 - val_accuracy: 0.8679 - val_loss: 0.3874
Epoch 2/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 54s 6s/step - accuracy: 0.8548 - loss: 0.3781 - val_accuracy: 0.8679 - val_loss: 0.3368
Epoch 3/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 54s 6s/step - accuracy: 0.8776 - loss: 0.3019 - val_accuracy: 0.8679 - val_loss: 0.2616
Epoch 4/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 82s 6s/step - accuracy: 0.8891 - loss: 0.2295 - val_accuracy: 0.8742 - val_loss: 0.2241
Epoch 5/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 82s 6s/step - accuracy: 0.9252 - loss: 0.1903 - val_accuracy: 0.9182 - val_loss: 0.1914


In [ ]:
test_loss, test_acc = model.evaluate(test_b)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.9037 - loss: 0.1784
Test Accuracy: 0.90


Normal vs defective prediction

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

def load_and_prepare_image(img_path, target_size=(160, 160)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # normalize like training
    img_array = np.expand_dims(img_array, axis=0)  # batch dimension
    return img_array

In [ ]:
img_path = '/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/test/Normal /01_open_circuit_02.jpg'
img_array = load_and_prepare_image(img_path)

prediction = model.predict(img_array)[0][0]  # sigmoid output
label = 'OK' if prediction > 0.5 else 'DEFECTIVE'

print(f"Predicted label: {label} (confidence: {prediction:.2f})")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/test/Normal /01_open_circuit_02.jpg'

In [ ]:
plt.imshow(image.load_img(img_path))
plt.title(f"Prediction: {label}")
plt.axis('off')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# === 1. Build the CNN Model ===
def build_binary_model():
    model = models.Sequential([
        layers.Input(shape=(160, 160, 3)),
        layers.Rescaling(1./255),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# === 2. Load Datasets (Using RGB Only, No Grayscale) ===
train_b = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/train',
    image_size=(160, 160),
    batch_size=32,
    label_mode='binary'
)

val_b = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/val',
    image_size=(160, 160),
    batch_size=32,
    label_mode='binary'
)

test_b = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/test',
    image_size=(160, 160),
    batch_size=32,
    label_mode='binary'
)

# === 3. Train the Model ===
model = build_binary_model()
history_b = model.fit(
    train_b,
    validation_data=val_b,
    epochs=15
)

# === 4. Evaluate on Test Set ===
test_loss, test_acc = model.evaluate(test_b)
print(f"Test Accuracy: {test_acc:.2f}")

# === 5. Load and Prepare a Single RGB Image for Prediction ===
def load_and_prepare_image(img_path, target_size=(160, 160)):
    img = image.load_img(img_path, color_mode='rgb', target_size=target_size)  # RGB mode
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# === 6. Predict a Single Image ===
img_path = '/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/test/Normal/13000021_temp.jpg'
img_array = load_and_prepare_image(img_path)

prediction = model.predict(img_array)[0][0]
label = 'NORMAL' if prediction > 0.5 else 'DEFECTIVE'

# === 7. Display Prediction Result ===
plt.imshow(image.load_img(img_path))  # Show in RGB
plt.title(f"Prediction: {label} (Confidence: {prediction:.2f})")
plt.axis('off')
plt.show()


converted every image to greyscale

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# === 1. Build the CNN Model ===
def build_binary_model():
    model = models.Sequential([
        layers.Input(shape=(160, 160, 3)),
        layers.Rescaling(1./255),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# === 2. Preprocessing: Convert RGB to Grayscale and back to 3-channel ===
@tf.function
def convert_to_grayscale(image, label):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.grayscale_to_rgb(image)  # Convert back to 3 channels
    return image, label

# === 3. Load Datasets (example) ===
# Replace with your own dataset loading
train_b = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/train',
    image_size=(160, 160),
    batch_size=32,
    label_mode='binary'
)
val_b = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/val',
    image_size=(160, 160),
    batch_size=32,
    label_mode='binary'
)
test_b = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/test',
    image_size=(160, 160),
    batch_size=32,
    label_mode='binary'
)

# === 4. Apply Grayscale Preprocessing to Datasets ===
train_b = train_b.map(convert_to_grayscale)
val_b = val_b.map(convert_to_grayscale)
test_b = test_b.map(convert_to_grayscale)

# === 5. Train the Model ===
model = build_binary_model()
history_b = model.fit(
    train_b,
    validation_data=val_b,
    epochs=15
)

# === 6. Evaluate on Test Set ===
test_loss, test_acc = model.evaluate(test_b)
print(f"Test Accuracy: {test_acc:.2f}")

# === 7. Load and Prepare a Single Image for Prediction ===
def load_and_prepare_image(img_path, target_size=(160, 160)):
    img = image.load_img(img_path, color_mode='grayscale', target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.repeat(img_array, 3, axis=-1)  # Make it 3-channel
    img_array = img_array / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# === 8. Predict a Single Image ===
img_path = '/content/drive/MyDrive/PCB_PROJECT_BINARY_SPLIT/test/Normal/13000021_temp.jpg'
img_array = load_and_prepare_image(img_path)

prediction = model.predict(img_array)[0][0]
label = 'NORMAL' if prediction > 0.5 else 'DEFECTIVE'

# === 9. Display Prediction Result ===
plt.imshow(image.load_img(img_path, color_mode='grayscale'), cmap='gray')
plt.title(f"Prediction: {label} (Confidence: {prediction:.2f})")
plt.axis('off')
plt.show()


Multiclass classifier

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get true labels from train_m dataset
y_train = np.concatenate([y.numpy() for x, y in train_m])

# Compute class weights
weights = compute_class_weight(class_weight='balanced',
                                classes=np.unique(y_train),
                                y=y_train)

class_weights = dict(enumerate(weights))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Automatically get number of classes from dataset
num_classes = 6
img_height, img_width = 128, 128  # match your dataset size

# Define CNN model
model_m = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')  # softmax for multiclass
])


In [ ]:
model_m.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # sparse categorical because labels are integers
              metrics=['accuracy'])


In [ ]:
history = model_m.fit(
    train_m,
    validation_data=val_m,
    epochs=20
)


In [ ]:
test_loss, test_acc = model_m.evaluate(test_b)
print(f"Test Accuracy: {test_acc:.2f}")

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Step 1: Get true labels and predictions
y_true = []
y_pred = []

for images, labels in test_m:
    preds = model_m.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))  # Choose class with highest probability

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Step 2: Confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=multiclass_class_names)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix")
plt.show()

# Step 3: Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=multiclass_class_names))


In [ ]:
!unzip /content/drive/MyDrive/archive.zip -d /content

In [ ]:
 !git clone https://github.com/ultralytics/yolov5.git
 !ls

In [ ]:
%cd yolov5
!pip install -r requirements.txt

In [ ]:
!python train.py --img 416 --batch 16 --epochs 20 --data data.yaml --weights yolov5s.pt --cache --name pcb_data